# 🤖 Human Interruption Agent Development

Build a LangGraph agent with human-in-the-loop approval capabilities for high-risk operations on Databricks.

**Tech Stack:** 
- 🏢 **Databricks Foundation Models** - Pay-per-token LLM endpoints
- 🗄️ **Lakebase (Postgres)** - Persistent state management
- 🔄 **LangGraph** - Stateful agent workflows with interruption points
- 📊 **MLflow** - Model tracking and deployment

## 📦 Install Requirements

Install all necessary packages for agent development and deployment.

In [0]:
# Install required packages from requirements.txt
%pip install -q -r ../requirements.txt

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
dbutils.library.restartPython()

## ⚙️ Configuration & Setup

Configure your Databricks environment and credentials. The widgets below allow you to customize:
- Lakebase instance for persistence
- Service principal credentials for authentication
- Model serving endpoint selection
- Unity Catalog location for functions and models

In [0]:
# Import all required libraries
import os
import sys
import pathlib
import uuid
import json
import time
from datetime import datetime
from sqlalchemy import create_engine, text
from databricks.sdk import WorkspaceClient
import mlflow
import mlflow.pyfunc
from mlflow.models.resources import DatabricksServingEndpoint

# Add src directory to Python path
repo_root = pathlib.Path().absolute()
src_path = repo_root / 'src'
if str(src_path) not in sys.path:
    sys.path.append(str(src_path))

# Import helper modules first
# from src.utils.helpers import read_requirements

# Create configuration widgets
dbutils.widgets.text("lakebase_instance", "agent-human-interaction", "Lakebase Instance Name")
dbutils.widgets.text("db_client_id", "<uuid>", "Databricks Client ID") #make sure this is created
dbutils.widgets.text("db_client_secret", "Secret", "Databricks Client Secret") #make sure this is created
dbutils.widgets.text("endpoint_name", "databricks-claude-3-7-sonnet", "Model Serving Endpoint")
dbutils.widgets.text("catalog_name", "_databricks_demos", "Databricks Catalog")
dbutils.widgets.text("schema_name", "sql_workshop", "Schema name")

# Get configuration from widgets
LAKEBASE_INSTANCE = dbutils.widgets.get("lakebase_instance")
DATABRICKS_CLIENT_ID = dbutils.widgets.get("db_client_id")
DATABRICKS_CLIENT_SECRET = dbutils.widgets.get("db_client_secret")
ENDPOINT_NAME = dbutils.widgets.get("endpoint_name")
WORKSPACE_URL = dbutils.notebook.entry_point.getDbutils().notebook().getContext().browserHostName().get()
CATALOG_NAME = dbutils.widgets.get("catalog_name")
SCHEMA_NAME = dbutils.widgets.get("schema_name")

# Set environment variables for AGENT initialization
os.environ["ENDPOINT_NAME"] = ENDPOINT_NAME
os.environ["LAKEBASE_INSTANCE"] = LAKEBASE_INSTANCE
os.environ["LAKEBASE_USER"] = DATABRICKS_CLIENT_ID
os.environ["DATABRICKS_CLIENT_ID"] = DATABRICKS_CLIENT_ID
os.environ["DATABRICKS_CLIENT_SECRET"] = DATABRICKS_CLIENT_SECRET
os.environ["DATABRICKS_HOST"] = f"http://{WORKSPACE_URL}"

print(f"✅ Environment variables set:")
print(f"   LAKEBASE_INSTANCE: {LAKEBASE_INSTANCE}")
print(f"   ENDPOINT_NAME: {ENDPOINT_NAME}")
print(f"   CATALOG_NAME: {CATALOG_NAME}")
print(f"   SCHEMA_NAME: {SCHEMA_NAME}")
print(f"   DATABRICKS_CLIENT_ID: {DATABRICKS_CLIENT_ID}")

In [0]:
spark.sql(f"""CREATE OR REPLACE FUNCTION {CATALOG_NAME}.{SCHEMA_NAME}.get_top_usage(
  target_date DATE
)
RETURNS TABLE (
  workspace_id BIGINT,
  sku_name STRING,
  usage_type STRING,
  total_usage DOUBLE,
  usage_unit STRING,
  estimated_cost DOUBLE
)
LANGUAGE SQL
COMMENT 'Returns top usage for a given day'
RETURN
  SELECT 
    u.workspace_id,
    u.sku_name,
    u.usage_type,
    SUM(u.usage_quantity) as total_usage,
    MAX(u.usage_unit) as usage_unit,
    SUM(u.usage_quantity * COALESCE(p.pricing.default, 0)) as estimated_cost
  FROM system.billing.usage u
  LEFT JOIN system.billing.list_prices p
    ON u.cloud = p.cloud
    AND u.sku_name = p.sku_name
    AND u.usage_start_time >= p.price_start_time
    AND (p.price_end_time IS NULL OR u.usage_start_time < p.price_end_time)
  WHERE 
    u.usage_date = target_date
  GROUP BY 
    u.workspace_id,
    u.sku_name,
    u.usage_type
  ORDER BY estimated_cost DESC
  LIMIT 100;
""")
spark.sql(f"GRANT EXECUTE ON FUNCTION {CATALOG_NAME}.{SCHEMA_NAME}.get_top_usage TO `{DATABRICKS_CLIENT_ID}`;")
spark.sql(f"GRANT CREATE MODEL ON SCHEMA {CATALOG_NAME}.{SCHEMA_NAME} TO `{DATABRICKS_CLIENT_ID}`")
spark.sql(f"GRANT CREATE TABLE ON SCHEMA {CATALOG_NAME}.{SCHEMA_NAME} TO `{DATABRICKS_CLIENT_ID}`")
spark.sql(f"GRANT USE CATALOG ON CATALOG {CATALOG_NAME} TO `{DATABRICKS_CLIENT_ID}`")
spark.sql(f"GRANT USE SCHEMA ON SCHEMA {CATALOG_NAME}.{SCHEMA_NAME} TO `{DATABRICKS_CLIENT_ID}`")
os.environ["UC_FUNCTIONS"] = f"{CATALOG_NAME}.{SCHEMA_NAME}.get_top_usage"

In [0]:
test = spark.sql(f"SELECT sku_name, usage_type, total_usage, usage_unit FROM {CATALOG_NAME}.{SCHEMA_NAME}.get_top_usage(CURRENT_DATE() - 1) LIMIT 5")
display(test)

sku_name,usage_type,total_usage,usage_unit
PREMIUM_SERVERLESS_REAL_TIME_INFERENCE_US_EAST_2,COMPUTE_TIME,6876.792172237659,DBU
PREMIUM_SERVERLESS_REAL_TIME_INFERENCE_AUSTRALIA_EAST,COMPUTE_TIME,3428.57142864,DBU
PREMIUM_SERVERLESS_REAL_TIME_INFERENCE_US_EAST,COMPUTE_TIME,3867.3599999999997,DBU
PREMIUM_SERVERLESS_REAL_TIME_INFERENCE_US_EAST,COMPUTE_TIME,3428.571456,DBU
PREMIUM_SERVERLESS_SQL_COMPUTE_US_WEST_2,COMPUTE_TIME,304.04372222222224,DBU


## 🗄️ Lakebase (Postgres) Setup

Initialize the Lakebase connection and create tables for conversation persistence. This enables the agent to maintain state across interruptions and resume conversations seamlessly.

**Important:** Ensure your Databricks client ID has been assigned as a Postgres role with appropriate permissions on the Lakebase instance.

In [0]:
#@TODO: Will want to move this to an OAuth token refresher for the agent, or using Postgres username/pwd
from src.lakebase.database import LakebaseDatabase
lb_conn = LakebaseDatabase(host=f"http://{WORKSPACE_URL}")
conn_uri = lb_conn.initialize_connection(user=DATABRICKS_CLIENT_ID, instance_name=LAKEBASE_INSTANCE)

In [0]:
from langgraph.checkpoint.postgres import PostgresSaver
with PostgresSaver.from_conn_string(conn_uri) as checkpointer:
    checkpointer.setup()

## 🚀 Initialize the Agent

Import and initialize the human interruption agent. The agent will automatically configure itself using the environment variables set earlier.

In [0]:
# make sure you assign the databricks client id access databricks_superuser
from src.agents.human_interruption_agent import AGENT

## 🧪 Test Local Agent

Let's test the agent locally before deploying to ensure it's working correctly.

In [0]:
print("✅ Using pre-initialized AGENT")
print(f"Agent type: {type(AGENT).__name__}")

✅ Using pre-initialized AGENT
Agent type: LangGraphChatAgent


### 🔍 Query Agent Capabilities

First, let's ask the agent about its available tools and functions. It should reference our Unity Catalog functions.

In [0]:
# Set the agent as the current model for this session
mlflow.models.set_model(AGENT)

response = AGENT.predict({
    "messages": [
        {"role": "user", "content": "What capabilities do you have, specifically for tools and functions?"}
    ]
})

thread_id = response.custom_outputs.get('thread_id')
print(f"Thread ID: {thread_id}")

print("Agent Response (agent type: %s, response type: %s)" % (type(AGENT), type(response)))
print(response.messages[-1].content)

Thread ID: e217c9a9-91b8-4833-9aa0-471218264e51
Agent Response (agent type: <class 'src.agents.human_interruption_agent.LangGraphChatAgent'>, response type: <class 'mlflow.types.agent.ChatAgentResponse'>)
I have access to a specific tool that allows me to retrieve data about usage statistics:

- `marcin_databricks_demos__sql_workshop__get_top_usage`: This function returns top usage data for a specified date. It has one optional parameter:
  - `target_date`: A date parameter that can be specified to get usage data for a particular day

This tool appears to be related to a Databricks SQL workshop and can provide information about top usage metrics for a given date.

Is there something specific about usage data that you'd like me to help you with using this tool?


### 💾 Test Conversation Persistence

Verify that the agent remembers previous messages using the thread ID.

In [0]:
time.sleep(2)
followup_response = AGENT.predict(
    messages=[
        {"role": "user", "content": "What was my previous question?"}
    ],
    custom_inputs={"thread_id": thread_id}  # Pass the thread_id here
)
thread_id = followup_response.custom_outputs.get('thread_id')
print("Agent Response (agent type: %s, response type: %s)" % (type(AGENT), type(followup_response)))
print(followup_response.messages[-1].content)

Agent Response (agent type: <class 'src.agents.human_interruption_agent.LangGraphChatAgent'>, response type: <class 'mlflow.types.agent.ChatAgentResponse'>)
Based on the conversation history available to me, your previous question was: "What capabilities do you have, specifically for tools and functions?"

In that question, you were asking me about what tools and functions I have access to, and I responded by explaining that I have access to the `marcin_databricks_demos__sql_workshop__get_top_usage` function which can retrieve top usage data for a specified date.


### ✅ Double-Check Memory

Ensure persistence is working correctly with another follow-up.

In [0]:
time.sleep(5) #Sleep to allow for persistence of previous query to occur.
followup_response2 = AGENT.predict(
    messages=[
        {"role": "user", "content": "What was my most recent question?"}
    ],
    custom_inputs={"thread_id": thread_id}  # Pass the thread_id here
)
thread_id = followup_response2.custom_outputs.get('thread_id')
print("Agent Response (agent type: %s, response type: %s)" % (type(AGENT), type(followup_response)))
print(followup_response2.messages[-1].content)

Agent Response (agent type: <class 'src.agents.human_interruption_agent.LangGraphChatAgent'>, response type: <class 'mlflow.types.agent.ChatAgentResponse'>)
Based on the conversation history available to me, your most recent question was: "What was my previous question?"

In response to that, I informed you that your previous question had been "What capabilities do you have, specifically for tools and functions?"


### 🛠️ Test Unity Catalog Function Usage

Trigger the agent to use our custom UC function - this should prompt for approval.

In [0]:
time.sleep(5)
tool_response = AGENT.predict(
    messages=[
        {"role": "user", "content": "Can you get me the top usage for 2025-09-17 on the platform?"}
    ],
    custom_inputs={"thread_id": thread_id}
)
thread_id = tool_response.custom_outputs.get('thread_id')
print("Agent Response (agent type: %s, response type: %s)" % (type(AGENT), type(tool_response)))
print(tool_response.messages[-1].content)

Agent Response (agent type: <class 'src.agents.human_interruption_agent.LangGraphChatAgent'>, response type: <class 'mlflow.types.agent.ChatAgentResponse'>)
📋 **Tool Approval Required**

I'm ready to execute the following tool:
• Function: `marcin_databricks_demos__sql_workshop__get_top_usage`
• Arguments: `{"target_date": "2025-09-17"}`

Please approve this action to continue, or provide alternative instructions.


### ✅ Approve and Resume

The agent has paused for approval. Let's approve the tool execution and see the results.

In [0]:
time.sleep(5)
resume_response = AGENT.resume(
    command_value="approved",  # or just None for default approval
    thread_id=thread_id
)

print("Resume Response:")
print(resume_response.messages[-1].content)
print(f"Interrupted: {resume_response.custom_outputs.get('is_interrupted')}")

/local_disk0/.ephemeral_nfs/envs/pythonEnv-47310911-28a1-4466-a7c3-0067ecf417c5/lib/python3.10/site-packages/unitycatalog/ai/core/databricks.py:600: UserWarning: The following parameters do not have descriptions: target_date for the function marcin_databricks_demos.sql_workshop.get_top_usage. Using Unity Catalog functions that do not have parameter descriptions limits the functionality for an LLM to understand how to call your function. To improve tool calling accuracy, provide verbose parameter descriptions that fully explain what the expected usage of the function arguments are.
  check_function_info(function_info)


Resume Response:
Based on the data retrieved for September 17, 2025, here are the top usage statistics:

The highest usage for that day is from workspace ID 1720970340056130 using PREMIUM_SERVERLESS_REAL_TIME_INFERENCE_US_EAST_2 with a total usage of 6,876.79 DBUs (Databricks Units) and an estimated cost of $481.38.

Other notable high usage entries include:
- Workspace 1595657218862477: 3,428.57 DBUs ($301.71) for PREMIUM_SERVERLESS_REAL_TIME_INFERENCE_AUSTRALIA_EAST
- Workspace 1756146239551418: 3,867.36 DBUs ($270.72) for PREMIUM_SERVERLESS_REAL_TIME_INFERENCE_US_EAST
- Workspace 658946740795418: 3,428.57 DBUs ($240.00) for PREMIUM_SERVERLESS_REAL_TIME_INFERENCE_US_EAST

The data shows that serverless real-time inference services across different regions are the highest usage categories, followed by SQL compute and all-purpose compute services.

Would you like me to provide any specific analysis of this usage data?
Interrupted: False


## 📝 Register Agent with MLflow

Register the agent to MLflow for tracking and deployment. The agent includes all necessary dependencies and configuration.

**Key Points:**
- Agent must call `mlflow.models.set_model()` for deployment compatibility
- Dependencies are bundled via `code_paths` parameter
- Resources include the Databricks serving endpoint configuration

In [0]:
# Configure MLflow 3 with Unity Catalog
agent_name = "human-interruption-agent"
mlflow.set_experiment(f"/Shared/{agent_name}")
mlflow.set_registry_uri('databricks-uc')

# Define agent_path variable properly
agent_path = "../src/agents/human_interruption_agent.py"

# Create agent configuration following the triager pattern
agent_config = {
    "endpoint_name": ENDPOINT_NAME,
    "temperature": 0.1,
    "max_tokens": 1000,
}

# Create resources list with DatabricksServingEndpoint
resources = [
    DatabricksServingEndpoint(endpoint_name=agent_config.get("endpoint_name"))
] if agent_config.get("endpoint_name") else []

# Define input example
input_example = {
    "messages": [
        {"role": "user", "content": "What capabilities do you have, specifically for tools and functions?"}
    ]
}

# Read requirements using the helper function
with open('../requirements.txt', 'r') as file:
    requirements=[x for x in file.read().splitlines() if len(x) > 0 and x[0] != '#']

# Log the agent as an MLflow model WITHOUT mlflow.start_run()
# This follows the exact pattern from dbx-agent-triager
model_info = mlflow.pyfunc.log_model(
    name=agent_name,
    python_model=agent_path,
    model_config=agent_config,
    resources=resources,
    input_example=input_example,
    pip_requirements=requirements,
    code_paths=["../src"]
)

print(f"✅ Agent logged to MLflow with Unity Catalog")
print(f"   Model URI: {model_info.model_uri}")
print(f"   Artifact Path: {model_info.artifact_path}")
print(f"   Registry URI: {mlflow.get_registry_uri()}")
print(f"   Resources: {resources}")
print(f"   Agent Config: {agent_config}")

# Store model info for later use
MODEL_URI = model_info.model_uri

🔗 View Logged Model at: https://adb-2482403303441070.10.azuredatabricks.net/ml/experiments/153332696905207/models/m-8c19e03a0a124f2c9fefe901630f4699?o=2482403303441070
/local_disk0/.ephemeral_nfs/envs/pythonEnv-47310911-28a1-4466-a7c3-0067ecf417c5/lib/python3.10/site-packages/databricks/connect/session.py:454: UserWarning: Ignoring the default notebook Spark session and creating a new Spark Connect session. To use the default notebook Spark session, use DatabricksSession.builder.getOrCreate() with no additional parameters.
  warnings.warn(new_notebook_session_msg)
2025/09/18 03:25:29 INFO mlflow.pyfunc: Predicting on input example to validate output
/local_disk0/.ephemeral_nfs/envs/pythonEnv-47310911-28a1-4466-a7c3-0067ecf417c5/lib/python3.10/site-packages/databricks/connect/session.py:454: UserWarning: Ignoring the default notebook Spark session and creating a new Spark Connect session. To use the default notebook Spark session, use DatabricksSession.builder.getOrCreate() with no addi

✅ Agent logged to MLflow with Unity Catalog
   Model URI: models:/m-8c19e03a0a124f2c9fefe901630f4699
   Artifact Path: dbfs:/databricks/mlflow-tracking/153332696905207/logged_models/m-8c19e03a0a124f2c9fefe901630f4699/artifacts
   Registry URI: databricks-uc
   Resources: [<mlflow.models.resources.DatabricksServingEndpoint object at 0x7f9477e22b30>]
   Agent Config: {'endpoint_name': 'databricks-claude-3-7-sonnet', 'temperature': 0.1, 'max_tokens': 1000}


### 🔍 Verify Model Registration

Quick sanity check to ensure the model deserializes correctly from MLflow.

In [0]:
os.environ['MLFLOW_ARTIFACT_UPLOAD_DOWNLOAD_TIMEOUT'] = '600'  # Timeout in seconds
loaded_model = mlflow.pyfunc.load_model(MODEL_URI)

# Run prediction on input_example
predictions = loaded_model.predict(input_example)

# Display the predictions
predictions

/local_disk0/.ephemeral_nfs/envs/pythonEnv-47310911-28a1-4466-a7c3-0067ecf417c5/lib/python3.10/site-packages/databricks/connect/session.py:454: UserWarning: Ignoring the default notebook Spark session and creating a new Spark Connect session. To use the default notebook Spark session, use DatabricksSession.builder.getOrCreate() with no additional parameters.
  warnings.warn(new_notebook_session_msg)


{'messages': [{'role': 'user',
   'content': 'What capabilities do you have, specifically for tools and functions?',
   'id': '599434af-e674-439b-85c2-dfd1ff793f85'},
  {'role': 'assistant',
   'content': 'I have access to a specific tool that allows me to retrieve data about usage patterns:\n\n- `marcin_databricks_demos__sql_workshop__get_top_usage`: This function returns top usage data for a specified date. It has one optional parameter called "target_date" which can be provided as a date string.\n\nThis tool appears to be related to a SQL workshop in Databricks and can help analyze usage patterns for a given day. If you\'d like to use this tool, I can help you retrieve the data by executing the function with your specified date.\n\nIs there a particular date you\'d like to analyze the top usage for?',
   'id': 'run--1c4f7659-d16b-4d81-848b-e8ff199ba546-0'}],
 'custom_outputs': {'thread_id': '48669a3c-01bc-42a3-a8d1-c3c5aa7991b4',
  'is_interrupted': False,
  'interrupt_data': None,


## 📊 Evaluations

Future section for MLflow evaluation sets with tool usage metrics.

In [0]:
#@TODO, include evaluation set with MLflow for tool usage.

## 🎯 Register to Unity Catalog

Register the model in Unity Catalog for centralized management and versioning.

In [0]:
# Unity Catalog location
uc_model_fqn = f"{CATALOG_NAME}.{SCHEMA_NAME}.{agent_name}"

# Register the model to the Unity Catalog
uc_registered_model_info = mlflow.register_model(model_uri=model_info.model_uri, name=uc_model_fqn)

import mlflow
from mlflow import MlflowClient

# Initialize the MLflow client
client = MlflowClient()

# Set the alias for the registered model
client.set_registered_model_alias(
    name=uc_model_fqn,
    alias="newest",
    version=uc_registered_model_info.version
)

Registered model 'marcin_databricks_demos.sql_workshop.human-interruption-agent' already exists. Creating a new version of this model...
2025/09/18 03:25:46 WARNING mlflow.store._unity_catalog.registry.rest_store: Unable to fetch model version's source run (with ID ) from tracking server. The source run may be deleted or inaccessible to the current user. No run link will be recorded for the model version.
2025/09/18 03:25:46 WARNING mlflow.store._unity_catalog.registry.rest_store: Unable to get model version source run's workspace ID from request headers. No run link will be recorded for the model version


Uploading artifacts:   0%|          | 0/18 [00:00<?, ?it/s]

🔗 Created version '9' of model 'marcin_databricks_demos.sql_workshop.human-interruption-agent': https://adb-2482403303441070.10.azuredatabricks.net/explore/data/models/marcin_databricks_demos/sql_workshop/human-interruption-agent/version/9?o=2482403303441070


### 📦 Load from Unity Catalog

Verify we can load the model from UC using the alias.

In [0]:
import mlflow

#@TODO: Implement MLflow tracing and MLFlow UI
#uc_model_fqn = f"databricks_agent.core.agent_builder"
model_uri = f"models:/{uc_model_fqn}@newest"
loaded_model = mlflow.pyfunc.load_model(model_uri)
loaded_model.predict(input_example)

/local_disk0/.ephemeral_nfs/envs/pythonEnv-47310911-28a1-4466-a7c3-0067ecf417c5/lib/python3.10/site-packages/databricks/connect/session.py:454: UserWarning: Ignoring the default notebook Spark session and creating a new Spark Connect session. To use the default notebook Spark session, use DatabricksSession.builder.getOrCreate() with no additional parameters.
  warnings.warn(new_notebook_session_msg)


{'messages': [{'role': 'user',
   'content': 'What capabilities do you have, specifically for tools and functions?',
   'id': 'e2ecbf73-615d-4150-96a3-1b9fc365b87c'},
  {'role': 'assistant',
   'content': "I have access to a specific tool that allows me to retrieve top usage data for a given day from what appears to be a Databricks SQL workshop environment. The function is called `marcin_databricks_demos__sql_workshop__get_top_usage` and it takes an optional parameter called `target_date`.\n\nThis function likely returns usage metrics or statistics for the specified date, which could include information about SQL queries, resource consumption, or user activity within a Databricks environment.\n\nIf you'd like me to use this tool to retrieve usage data for a specific date, I can do that for you. You would just need to specify which date you're interested in, and I'll prepare the function call for your approval before executing it.",
   'id': 'run--fb66e09c-4f6e-4644-a01c-2be99ed1ec43-0'

## 🚀 Deploy Agent to Production

Deploy the agent as a Model Serving endpoint for production use. This creates a REST API and Review App interface.

In [0]:
from mlflow.utils import databricks_utils as du
from mlflow.models.resources import DatabricksServingEndpoint, DatabricksVectorSearchIndex

def parse_deployment_info(deployment_info):
  browser_url = du.get_browser_hostname()
  message = f"""Deployment of {deployment_info.model_name} version {deployment_info.model_version} initiated.  This can take up to 15 minutes and the Review App & REST API will not work until this deployment finishes. 

  View status: https://{browser_url}/ml/endpoints/{deployment_info.endpoint_name}
  Review App: {deployment_info.review_app_url}"""
  return message

### 🔐 Configure Environment Variables

Set all required environment variables for the deployed agent to function properly.

In [0]:
envvars = {
    "ENDPOINT_NAME" : ENDPOINT_NAME,
    "LAKEBASE_INSTANCE" : LAKEBASE_INSTANCE,
    "LAKEBASE_USER" : DATABRICKS_CLIENT_ID,
    "DATABRICKS_CLIENT_ID" : DATABRICKS_CLIENT_ID,
    "DATABRICKS_CLIENT_SECRET" : DATABRICKS_CLIENT_SECRET,
    "DATABRICKS_HOST" : f"http://{WORKSPACE_URL}",
    "UC_FUNCTIONS" : f"{CATALOG_NAME}.{SCHEMA_NAME}.get_top_usage",
}

In [0]:
from databricks import agents
# Deploy to enable the Review APP and create an API endpoint
# Note: scaling down to zero will provide unexpected behavior for the chat app. Set it to false for a prod-ready application.
deployment_info = agents.deploy(uc_model_fqn, model_version=uc_registered_model_info.version, scale_to_zero=True,
    resources=resources,
    #@TODO: implement secrets
    #environment_vars={"DATABRICKS_HOST":"https://e2-demo-field-eng.cloud.databricks.com", "DATABRICKS_TOKEN":"{{secrets/marcin_jimenez/databricks_token}}"}
    environment_vars=envvars,
)

# instructions_to_reviewer = f"""## Instructions for Testing the Databricks Documentation Assistant chatbot

# Your inputs are invaluable for the development team. By providing detailed feedback and corrections, you help us fix issues and improve the overall quality of the application. We rely on your expertise to identify any gaps or areas needing enhancement."""

# # Add the user-facing instructions to the Review App
# agents.set_review_instructions(uc_model_fqn, instructions_to_reviewer)

print(parse_deployment_info(deployment_info))


    Deployment of marcin_databricks_demos.sql_workshop.human-interruption-agent version 9 initiated.  This can take up to 15 minutes and the Review App & Query Endpoint will not work until this deployment finishes.

    View status: https://adb-2482403303441070.10.azuredatabricks.net/ml/endpoints/agents_marcin_databricks_demos-sql_workshop-human-interruption
    Review App: https://adb-2482403303441070.10.azuredatabricks.net/ml/review-v2/0633ac30f9554b02b367cad177dda356/chat

You can refer back to the links above from the endpoint detail page at https://adb-2482403303441070.10.azuredatabricks.net/ml/endpoints/agents_marcin_databricks_demos-sql_workshop-human-interruption.
Deployment of marcin_databricks_demos.sql_workshop.human-interruption-agent version 9 initiated.  This can take up to 15 minutes and the Review App & REST API will not work until this deployment finishes. 

  View status: https://adb-2482403303441070.10.azuredatabricks.net/ml/endpoints/agents_marcin_databricks_demos-

## ✨ Test Production Deployment

Verify the deployed endpoint is working correctly by making a test prediction.

In [0]:
from mlflow.deployments import get_deploy_client

deploy_client = get_deploy_client()

# Query your deployed agent
response = deploy_client.predict(
    endpoint=deployment_info.endpoint_name,
    inputs=input_example
)

print(response.messages[-1]['content'])

I have access to a specific tool that allows me to retrieve data about usage patterns:

- `marcin_databricks_demos__sql_workshop__get_top_usage`: This function returns top usage data for a specified date. It has one optional parameter called "target_date" which can be provided as a date string.

This tool appears to be related to a Databricks SQL workshop and can help analyze usage patterns for a particular date. If you'd like me to use this tool to retrieve data, you can specify a date, and I can execute the query for you after receiving your approval.

Is there a specific date for which you'd like to see top usage data?
